In [1]:
import numpy as np
import pygrib
import s3fs
import os
from datetime import timedelta, date

startDate = date(2000, 1, 1)
#endDate = date(2019, 2, 18)
endDate = date(2000, 1, 13)
    
directory = os.getcwd()

# Predetermined pixel for Alta, lines below can find for other locations
i,j = 198,993
desiredLat = 40.57
desiredLon = -111.67
#grbs = pygrib.open('exampleGefsFile.grib2')
#desiredLon = 360 + desiredLon
#lats, lons = grbs[1].latlons()
#a = abs(lats - float(desiredLat)) + abs(lons - float(desiredLon))
#i,j = np.unravel_index(a.argmin(), a.shape)

# Use the anonymous credentials to access public data
fs = s3fs.S3FileSystem(anon=True)

delta = timedelta(days=1)
while startDate < endDate:
    startDate += delta
    
    # working command
    # fs.ls('noaa-gefs-retrospective/GEFSv12/reforecast/2000/2000010100/c00/Days:1-10/') 
    bucketUrl = 'noaa-gefs-retrospective/GEFSv12/reforecast/'
    year = str(startDate.year)
    month = str(startDate.month).zfill(2)
    day = str(startDate.day).zfill(2)
    hour = '00'
    date = year + month + day + hour

    f = open('gefsReanlysis.csv', 'r')
    contents = f.read()
    f.close()
    if contents.find(date) != -1:
        continue
    
    print('working on ' + str(startDate))
    
    ensMember = 'c00'
    range = 'Days:1-10/'
    fileSuffix = '_' + ensMember + '.grib2'
    uFile = 'ugrd_hgt_' + date + fileSuffix
    vFile = 'vgrd_hgt_' + date + fileSuffix
    gustFile = 'gust_sfc_' + date + fileSuffix

    fs.get(bucketUrl + year + '/' + date + '/' + ensMember + '/' + range + uFile, uFile)
    fs.get(bucketUrl + year + '/' + date + '/' + ensMember + '/' + range + vFile, vFile)
    fs.get(bucketUrl + year + '/' + date + '/' + ensMember + '/' + range + gustFile, gustFile)

    ugrbs = pygrib.open(uFile)
    vgrbs = pygrib.open(vFile)
    gustGrbs = pygrib.open(gustFile)

    output = date + ','

    index = 1
    for ugrb in ugrbs:
        strName = str(ugrb)
        #print(strName)
        if 'level 10 m' in strName:
            fcstTime = strName.find('fcst time ') + len('fcst time ')
            fhr = strName[fcstTime:strName.find('hrs',fcstTime)].strip(' ')
            if int(fhr) > 48:
                break
            uArray = ugrb.values
            vArray = vgrbs[index].values
            gust = gustGrbs[index].values
            gust = gust[i][j]
            output = output + fhr + ',' + '%.2f' % uArray[i][j] + ',' + '%.2f' % vArray[i][j]+ ',' + '%.2f' % gust + ','
        index += 1
    output = output[:-1]
    output = output + '\n'
    ugrbs.close()
    vgrbs.close()
    gustGrbs.close()

    os.remove(directory + '/' + uFile)
    os.remove(directory + '/' + vFile)
    os.remove(directory + '/' + gustFile)

    f = open('gefsReanlysis.csv', 'a')
    f.write(output)
    f.close()
